In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import re
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [3]:
!pip install opencv-contrib-python
!pip install --upgrade google-cloud-vision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.2/444.2 KB 27.2 MB/s eta 0:00:00


In [4]:
import os
import io
import numpy as np
import platform
import cv2
from google.cloud import vision

In [5]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/gdrive/MyDrive/Graduation-project/graduation-project-380601-f332a175dd99.json'
 
client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

In [6]:
#경로설정후 텍스트 값 받기
def get_text(path):
  # '/content/gdrive/MyDrive/Graduation-project/문제/102.PNG' 
  path = path
  with io.open(path, 'rb') as image_file:
      content = image_file.read()
  image = vision.Image(content=content)
 
  response = client.text_detection(image=image)
  texts = response.text_annotations
  #print(texts[0].description)
  return texts[0].description

In [7]:
#문단이랑 보기 내용 나누기
def split_other(other):
  other1 = other.replace("?",".")
  other2 = other1.replace("!",".")
  #other3 = other2.replace("\n"," ")
  other3 = other2.replace('"',"")
  temp = other3.split(".")
  #temp 는 문단 내용
  #for i in temp:
  #  print(i)
  answer = temp.pop()
  return temp,answer

In [8]:
#답안지 추출
def get_answer(answer):
  answer = re.sub(r"[0-9]", "", answer)
  answer = answer.replace("→","")
  #print(answer)

  # *있는거 제거 (단어 설명 파트)
  tmp = answer.split("\n")
  tmp = list(filter(None, tmp))
  #print(tmp)
  
  answer = []
  for i in tmp:
    #print(i)
    a = i.find("*")
    #print(a)
    if a == -1:
      #print("came in")
      answer.append(i)
  
  answer = " ".join(answer)
  answer = answer.strip()
  answer = re.sub(r"\s+", " ", answer)
  answer = answer.split(" ")
  answer = list(filter(None, answer))

  # and 제거
  remove_set = {'and'}
  answer = [i for i in answer if i not in remove_set]
  return answer

In [9]:
# 내용 정리
def get_main(main):
  temp = []
  for i in main:
    i = i.replace("\n"," ")
    temp.append(i)
  #print(i)
  return temp

In [10]:
"""
# '/content/gdrive/MyDrive/Graduation-project/문제/4.png' 
for i in range(1,101):
  path = '/content/gdrive/MyDrive/Graduation-project/문제/' + i '.png'
  all = get_text(path)
  question, other = all.split('?',maxsplit=1)

  #question은 질문
  #question에서 변화단어 유무로 구별

  mode = question.find("변화")
  if mode == -1:
    mode = 0
  else:
    mode = 1
  
  main, answer = split_other(other)
  main = get_main(main)
  answer = get_answer(answer)
"""

'\n# \'/content/gdrive/MyDrive/Graduation-project/문제/4.png\' \nfor i in range(1,101):\n  path = \'/content/gdrive/MyDrive/Graduation-project/문제/\' + i \'.png\'\n  all = get_text(path)\n  question, other = all.split(\'?\',maxsplit=1)\n\n  #question은 질문\n  #question에서 변화단어 유무로 구별\n\n  mode = question.find("변화")\n  if mode == -1:\n    mode = 0\n  else:\n    mode = 1\n  \n  main, answer = split_other(other)\n  main = get_main(main)\n  answer = get_answer(answer)\n'

In [11]:
path = '/content/gdrive/MyDrive/Graduation-project/문제/50.png'
all = get_text(path)
print(all)
question, other = all.split('?',maxsplit=1)

#question은 질문
#question에서 변화단어 유무로 구별

mode = question.find("변화")
if mode == -1:
  mode = 0
else:
  mode = 1

main, answer = split_other(other)
main = get_main(main)
answer = get_answer(answer)
for i in main:
  print(i)
print(answer)

19. 다음 글에 드러난 I'의 심경으로 가장 적절한 것은?
I'm leaving early tomorrow morning, finally! I've always
wanted to explore the Amazon, the unknown and mysterious
world. At this hour, the great Emerald Amazon Explorer should
be at the port waiting for me to get on board. Freshwater dolphins
will escort me on the playful river, and 500 species of birds, half
a dozen species of monkeys, and numerous colorful butterflies
will welcome me into their kingdom. I wish I could camp in the
wild and enjoy the company of mosquitos, snakes, and spiders.
I'd love to make the world's largest rainforest home. My heart
swells as much as my chubby bags; yet, I'd better get some sleep
since a long, tough journey is ahead of me.
3 frustrated
excited
4 indifferent
exhausted
relieved
 I'm leaving early tomorrow morning, finally
 I've always wanted to explore the Amazon, the unknown and mysterious world
 At this hour, the great Emerald Amazon Explorer should be at the port waiting for me to get on board
 Freshwater dolphin

In [12]:
! pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.0 MB/s eta 0:00:00


In [13]:
from transformers import pipeline

In [14]:
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [15]:
temp = []
if len(answer) == 10:
  temp.append(answer[0] + " " + answer[1])
  temp.append(answer[2] + " " + answer[3])
  temp.append(answer[4] + " " + answer[5])
  temp.append(answer[6] + " " + answer[7])
  temp.append(answer[8] + " " + answer[9])
elif len(answer) == 5:
  temp = answer
else:
  print("error")
print(temp)

['frustrated', 'excited', 'indifferent', 'exhausted', 'relieved']


In [16]:
# 문단 내용 분석
from collections import Counter
check = []
max = 0

for i in main:
  emotion_labels = emotion(i)
  #print(emotion_labels)
  if emotion_labels[0]['label'] != 'neutral':
    check.append(emotion_labels[0]['label'])
    if emotion_labels[0]['score'] > max:
      max = emotion_labels[0]['score']
      max_result = emotion_labels[0]['label']

#print(check)
cnt = Counter(check)
many = cnt.most_common(1)
#many[0][0] 최빈값 plus_result 전체 max_result 이 최고
print(many[0][0])
plus = " ".join(check)
print(plus)
plus_result = emotion(plus)
print(max_result)

desire
approval desire excitement joy desire optimism
desire


In [17]:
# 답안지 내용 분석
#print(temp)
for i in temp:
  emotion_labels = emotion(i)
  print(emotion_labels)
  if emotion_labels[0]['label'] == many[0][0]:
    print("최다")
    print(i)
  if emotion_labels[0]['label'] == plus_result:
    print("전체")
    print(i)
  if emotion_labels[0]['label'] == max_result:
    print("최고")
    print(i)

[{'label': 'disgust', 'score': 0.9915076494216919}]
[{'label': 'excitement', 'score': 0.992557168006897}]
[{'label': 'disapproval', 'score': 0.7316359281539917}]
[{'label': 'disgust', 'score': 0.9665487408638}]
[{'label': 'disgust', 'score': 0.3584071695804596}]
